<a href="https://colab.research.google.com/github/RyanKaplan999/graph-transfer/blob/main/cautiously_optimistic_with_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

def format_pytorch_version(version):
    return version.split('+')[0]

def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

In [2]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch_geometric

torch_geometric.__version__


'2.1.0'

Download our dataset:

In [4]:
from re import I
from torch_geometric.data import Data 

from torch_geometric.datasets import TUDataset

import torchvision.transforms as transforms

import numpy as np
import torch
import torchvision
import torch.optim as optim
import time
import torch.nn as nn
from torchvision import models
import matplotlib.pyplot as plt
import tensorflow as tf

dataset = TUDataset(root='/tmp/AIDS', name='AIDS')
dataset = dataset.shuffle()

# takes way too long
#dataset = TUDataset(root='/tmp/FRANKENSTEIN', name='FRANKENSTEIN') 
#dataset = dataset.shuffle()

# enzymes works ok (much fewer examples, but 6 classes) gets regularly above 20%, its at least better than guessing
#dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
#dataset = dataset.shuffle()

#this one has 100 classes
#dataset = TUDataset(root='/tmp/COIL-DEL', name='COIL-DEL')
#dataset = dataset.shuffle()

# to divide evenly, make sure it's divisible by 32
dataset = dataset[(len(dataset) % 32):]

print(dataset)
print("length of set", len(dataset))
print("# of classes", dataset.num_classes)

max = 0
for data in dataset:
  if len(data.x) > max:
    max = len(data.x)
print("Max size of adjacency matrix:", max, "x", max)


Extracting /tmp/AIDS/AIDS/AIDS.zip
Processing...
Done!


AIDS(1984)
length of set 1984
# of classes 2
Max size of adjacency matrix: 95 x 95


Construct all of our "images"

In [5]:

print("Initializing graph to image")

# convert our graph data to "images" containing the adjacency matrix
images = []

for j in range(len(dataset)):

    graph = dataset[j]

    adj_matrix = np.zeros([3, 224, 224])

    num_nodes = len(graph.x)
    num_edges = len(graph.edge_index[0])
    offset = int((224 - num_nodes) / 2)

    # for each edge, initialize its entry in the adjaceny matrix as a colored pixel
    for i in range(num_edges):
      x = graph.edge_index[0][i]
      y = graph.edge_index[1][i]
      adj_matrix[0][offset + x][offset + y] = 1
      adj_matrix[1][offset + x][offset + y] = 1
      adj_matrix[2][offset + x][offset + y] = 1

    images.append((torch.from_numpy(adj_matrix), graph.y.item()))

  # have to convert them to tensors so it actually works

# splits it 90/10 train/test
cutoff = int(len(images) * 0.9)
training_data = images[:cutoff]
test_data = images[cutoff:]

print(len(images), 'total examples')
print(len(training_data), "training examples")
print(len(test_data), "test examples")
 

Initializing graph to image
1984 total examples
1785 training examples
199 test examples


In [6]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')


Using cuda:0 device


Download our pretrained model:

In [7]:

import torch.nn.functional as F
from torch_geometric.nn import GCNConv

imgClassifier = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1').to(device)

In [8]:
# x is [1,3,224,224], mask is [4, 3, offset, offset]
def pad(x, mask):
  temp = x.numpy()
  for i in range(224):
    for j in range(224):
      
      if i < offset and j < offset:
        temp[0][0][i][j] = mask[0][0][i][j]
        temp[0][1][i][j] = mask[0][1][i][j]
        temp[0][2][i][j] = mask[0][2][i][j]

      if i > max + offset and j < offset:
        temp[0][0][i][j] = mask[1][0][i - (max + offset)][j]
        temp[0][1][i][j] = mask[1][1][i - (max + offset)][j]
        temp[0][2][i][j] = mask[1][2][i - (max + offset)][j]
      
      if i < offset and j > max + offset:
        temp[0][0][i][j] = mask[2][0][i][j - (max + offset)]
        temp[0][1][i][j] = mask[2][1][i][j - (max + offset)]
        temp[0][2][i][j] = mask[2][2][i][j - (max + offset)]
      
      if i > max + offset and j > max + offset:
        temp[0][0][i][j] = mask[3][0][i - (max + offset)][j - (max + offset)]
        temp[0][1][i][j] = mask[3][1][i - (max + offset)][j - (max + offset)]
        temp[0][2][i][j] = mask[3][2][i - (max + offset)][j - (max + offset)]
      
      else:
        temp[0][0][i][j] = x[0][0][i][j]
        temp[0][1][i][j] = x[0][1][i][j]
        temp[0][2][i][j] = x[0][2][i][j]
  return temp


x = np.zeros([1,3,224,224])
x = torch.tensor(x)
mask = np.ones([4, 3, offset, offset])
mask = torch.tensor(mask)

padded = pad(x, mask)

print("max: ", max)
print("offset:", offset)
print(padded[0][0][0][0])
print(padded[0][0][offset + 5][offset + 5])



max:  95
offset: 106
1.0
0.0


In [9]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # freeze ALL parameters
        for param in imgClassifier.parameters():
          param.requires_grad = False

        #self.W = torch.nn.Parameter(torch.tensor((1000, dataset.num_classes), dtype=float)) # output transformation parameter 
        self.W = torch.nn.Parameter(torch.nn.init.uniform_(torch.empty(1000, dataset.num_classes))).to(device)

        #print(self.W[0][0])

        #b = torch.nn.Parameter(torch.tensor((dataset.num_classes, 1), dtype=float))
        self.b = torch.nn.Parameter(torch.nn.init.uniform_(torch.empty(1, dataset.num_classes))).to(device)

        # a mask parameter for each corner of the padded image, and each channel; [corner, channel, width, height] with corner=[1..4] going from upper left clockwise to lower left.
        #Omega = torch.nn.Parameter(torch.tensor((4, 3, offset, offset), dtype=float)) 
        self.Omega = torch.nn.Parameter(torch.nn.init.uniform_(torch.empty(4, 3, offset, offset))).to(device)

    def forward(self, x):
        z = self.pad(x, self.Omega)
        #z = torch.unsqueeze(z, 0) # size is [1,3,224,224]

        z = z.float()
    
        out = imgClassifier(z) # size is [1,1000]

        reducedOut = torch.matmul(out, self.W.to(device))      # size is [1, 100]
        #reducedOut = torch.matmul(out, self.W)      # size is [1, 100]

        #return F.log_softmax((reducedOut + self.b.to(device)), dim=0)
        return F.log_softmax((reducedOut + self.b), dim=0)
    
    def pad(self, x, mask):
      # if x is [batchsize, channels, width, height], this pads x with the mask
      temp = x.cpu().numpy()
      for i in range(224):
        for j in range(224):
          
          if i < offset and j < offset:
            temp[0][0][i][j] = mask[0][0][i][j]
            temp[0][1][i][j] = mask[0][1][i][j]
            temp[0][2][i][j] = mask[0][2][i][j]

          if i > max + offset and j < offset:
            temp[0][0][i][j] = mask[1][0][i - (max + offset)][j]
            temp[0][1][i][j] = mask[1][1][i - (max + offset)][j]
            temp[0][2][i][j] = mask[1][2][i - (max + offset)][j]
          
          if i < offset and j > max + offset:
            temp[0][0][i][j] = mask[2][0][i][j - (max + offset)]
            temp[0][1][i][j] = mask[2][1][i][j - (max + offset)]
            temp[0][2][i][j] = mask[2][2][i][j - (max + offset)]
          
          if i > max + offset and j > max + offset:
            temp[0][0][i][j] = mask[3][0][i - (max + offset)][j - (max + offset)]
            temp[0][1][i][j] = mask[3][1][i - (max + offset)][j - (max + offset)]
            temp[0][2][i][j] = mask[3][2][i - (max + offset)][j - (max + offset)]
          
          else:
            temp[0][0][i][j] = x[0][0][i][j]
            temp[0][1][i][j] = x[0][1][i][j]
            temp[0][2][i][j] = x[0][2][i][j]

      return torch.tensor(temp).to(device)


# Example of applying to a single datapoint
classifier = GCN()

x = training_data[0][0] # x is a single input
x = torch.unsqueeze(x, 0)
x = x.float()

print(x.size())

yhat = classifier(x.to(device))
print(yhat[0,0])
#sum = 0
#for i in range(100):
#  sum += yhat[0][i]
#print(sum)
#print(torch.max(yhat))

torch.Size([1, 3, 224, 224])
tensor(0., device='cuda:0', grad_fn=<SelectBackward0>)


In [10]:
from torch.utils.data import DataLoader

# Train
model = GCN().to(device)

# DataLoaders will return randomly shuffled minibatches in each epoch
train_loader = DataLoader(training_data, batch_size=32, 
                              shuffle=True, num_workers=1)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=1)

# Choose the optimizer
#optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(imgClassifier.parameters(), lr=0.01, weight_decay=5e-4)

# set the loss function
criterion = nn.CrossEntropyLoss() 

for batchnum, (X,y) in enumerate(train_loader):
  if batchnum == 0:
    print(y.size())

torch.Size([32])


In [ ]:
# Does one epoch of training

train_loss_set , train_acc_set = [], []
val_loss_set , val_acc_set = [], []

def train_epoch(dataloader, model, criterion, optimizer):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    
    for batchnum, (X, y) in enumerate(dataloader):

        X = X.to(device=device)
        y = y.to(device)

        # Compute prediction and loss on the minibatch
        yhat = model(X) 
        loss = criterion(yhat, y)

        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (yhat.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    train_loss_set.append(train_loss)
    train_acc_set.append(100 * correct)

            
# Computes loss and accuracy on the validation set

def validate(dataloader, model, criterion):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): # we don't want to accumulate the gradients during validation
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            yhat = model(X)
            test_loss += criterion(yhat, y).item()
      
            correct += (yhat.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    val_loss_set.append(test_loss)
    val_acc_set.append(100 * correct)

start = time.time()
for t in range(10):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train() # turn model to train mode (for dropout, etc)
    train_epoch(train_loader, model, criterion, optimizer)
    model.eval() # turn model to evaluation mode (for dropout, etc)
    validate(test_loader, model, criterion)
    
print("Done!")

end = time.time()
print(round(((end-start)/60),1) , 'minutes')

print("Randomly guessing would be an accuracy of ", (1/dataset.num_classes) * 100)

# plot our accuracy and loss
plt.figure(figsize=(10, 7))
plt.plot(train_acc_set, color='green', label='train accuracy')
plt.plot(val_acc_set, color='blue', label='validataion accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

plt.figure(figsize=(10, 7))
plt.plot(train_loss_set, color='orange', label='train loss')
plt.plot(val_loss_set, color='red', label='validataion loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

Epoch 1
-------------------------------
Train Error: 
 Accuracy: 75.6%, Avg loss: 2.022582 

Test Error: 
 Accuracy: 73.4%, Avg loss: 1.711738 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 77.8%, Avg loss: 1.829175 

Test Error: 
 Accuracy: 78.9%, Avg loss: 1.251122 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 76.9%, Avg loss: 1.952076 

Test Error: 
 Accuracy: 73.4%, Avg loss: 1.836023 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 71.7%, Avg loss: 2.161892 

Test Error: 
 Accuracy: 68.8%, Avg loss: 2.137464 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 74.3%, Avg loss: 1.984501 

Test Error: 
 Accuracy: 76.9%, Avg loss: 1.194492 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 77.7%, Avg loss: 1.949192 

Test Error: 
 Accuracy: 78.9%, Avg loss: 1.433545 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 73.4%, Avg loss: 1.920691 

Test Error: 
 Accuracy: 66.3%, Avg lo